# GPS Time and UTC conversion

## Time - Introduction

In [2]:
import datetime

In [6]:
# Unix timestamp as a float
unix_ts = 1557261145.816266

In [7]:
# as datetime
date_time = datetime.datetime.fromtimestamp(unix_ts)
date_time

datetime.datetime(2019, 5, 7, 21, 32, 25, 816266)

In [8]:
# formatted. Specify format then call strftime on datetime object
fmt = "%Y-%m-%d, %H:%M:%S.%f"
date_time.strftime(fmt)

'2019-05-07, 21:32:25.816266'

In [9]:
# weekday 0 = Monday
weekday_number = date_time.date().weekday()
weekday_number

1

In [10]:
# converting datetime to and from strings
new_datetime = datetime.datetime.strptime('2018-05-07', '%Y-%m-%d')
new_datetime

datetime.datetime(2018, 5, 7, 0, 0)

In [11]:
datestr = new_datetime.strftime('%Y-%m-%d')
print(datestr)

2018-05-07


In [12]:
datestr = new_datetime.strftime('%d %b %Y')
print(datestr)

07 May 2018


In [13]:
datestr = new_datetime.strftime('%c')
print(datestr)

Mon May  7 00:00:00 2018


In [14]:
# lots of possibilities with Python's strftime directives
# http://strftime.org/
# https://pyformat.info/
now = datetime.datetime.now()
nowstr = now.strftime('%c; Week:%W Day:%j')
print(nowstr)

Sun Jun  2 19:59:43 2019; Week:21 Day:153


In [4]:
sshot = 'per_1564113757'
sshot_date_time = datetime.datetime.fromtimestamp(int(sshot.split('_')[1]))
sshot_date_time

datetime.datetime(2019, 7, 26, 5, 2, 37)

In [5]:
datestr = sshot_date_time.strftime('%Y %b %d')
print(datestr)

2019 Jul 26


In [6]:
timestr = sshot_date_time.strftime('%H:%M:%S')
print(timestr)

05:02:37


## Leap Seconds

As of May 2019, there hve been 18 leap seconds since the GPS Epoch, Jan 6, 1980. Leap seconds are not added by GPS. This means that GPS time is ahead of UTC by the current number of leap seconds.  

To convert UTC to GPS time, add the current leap seconds to UTC. To convert GPS time to UTC, subract leap seconds from GPS time.  

Adding a leap second effectively stops UTC for one second (to allow the earth's rotation to catch up). GPS time continues so it gets ahead of UTC by 1 second. [US Navy bulletin](ftp://tycho.usno.navy.mil/pub/gps/leapsecnanu2016.txt)

When a leap second is added, the UTC clock sequence is:  
 `31 DEC 2016 23 HOURS 59 MINUTES 59 SECONDS`  
 `31 DEC 2016 23 HOURS 59 MINUTES 60 SECONDS`  
 `01 JAN 2017 00 HOURS 00 MINUTES 00 SECONDS`  

In [15]:
# May 2019, leap seconds = 18
leapSec = 18

In [16]:
# GPS Epoch 1980-01-06 00:00:00
gpsEpoch = datetime.datetime(1980, 1, 6, 0, 0, 0)
gpsEpoch

datetime.datetime(1980, 1, 6, 0, 0)

In [17]:
leapSecDelta = datetime.timedelta(seconds=leapSec)
leapSecDelta

datetime.timedelta(seconds=18)

## Process to convert time to GPS time

In [18]:
# example time 2005-01-28 13:30:27.653187
#myTime = datetime.datetime(2019, 1, 28, 13, 30, 27, 653187)
myTime = datetime.datetime(2019, 5, 29, 14, 34)
myTime

datetime.datetime(2019, 5, 29, 14, 34)

In [19]:
# correct for leap seconds
myTime += leapSecDelta
myTime

datetime.datetime(2019, 5, 29, 14, 34, 18)

In [20]:
# GPS time is expressed as the number of weeks since the GPS Epoch and 
# the number of seconds in thsi week. (Week, ToW)
timeSinceGpsEpoch = myTime - gpsEpoch
timeSinceGpsEpoch

datetime.timedelta(days=14388, seconds=52458)

In [21]:
gpsWeek, days = divmod(timeSinceGpsEpoch.days, 7)

In [22]:
gpsWeek

2055

In [23]:
# ToW = (days this week * seconds per day) + seconds today + microseconds
gpsTow = (days * 86400) + timeSinceGpsEpoch.seconds + (timeSinceGpsEpoch.microseconds / 1e6)
gpsTow

311658.0

In [24]:
print(f'UTC = {myTime.strftime(fmt)} -> GPS = {gpsWeek}, {gpsTow}')

UTC = 2019-05-29, 14:34:18.000000 -> GPS = 2055, 311658.0


## Process to convert GPS time to UTC (naive)

In [224]:
gpsTime = (2038, 135045.653187)

In [225]:
gpsWeek, gpsTow = gpsTime

In [226]:
# extract microseconds - but floating point ops will introduce an error
gpsUsec = gpsTow - int(gpsTow)
gpsUsec

0.653186999988975

In [227]:
gpsWeekDays = gpsWeek * 7 

In [228]:
gpsTowDays, gpsTowSeconds = divmod(gpsTow, 86400) 

In [229]:
gpsDays = int(gpsWeekDays + gpsTowDays)
gpsDays

14267

In [230]:
gpsSeconds = int(gpsTowSeconds)
gpsSeconds

48645

In [231]:
gpsMicroseconds = int((gpsTowSeconds - gpsSeconds) * 1e6)
gpsMicroseconds

653186

In [232]:
utcTime = gpsEpoch + datetime.timedelta(days=gpsDays, seconds=gpsSeconds, microseconds=gpsMicroseconds)

In [233]:
# correct for leap seconds
utcTime -= leapSecDelta
utcTime

datetime.datetime(2019, 1, 28, 13, 30, 27, 653186)

In [234]:
print(utcTime.strftime(fmt))

2019-01-28, 13:30:27.653186


## Functions

In [235]:
def datetime_to_gps(dt, ls):
    '''Convert a date time to GPS time.
    GPS time does not include leap seconds.
    Leap seconds must be added to date time before conversion.
    
    Receives datetime object, leap seconds
    Returns GPS time in Weeks since GPS Epoch and Seconds of current week.
    
    '''
    
    gpsEpoch = datetime.datetime(1980, 1, 6, 0, 0, 0, 0)
    
    # adjust given date time for leap seonds 
    dt += datetime.timedelta(seconds=ls)
    
    timeSinceGpsEpoch = dt - gpsEpoch
    
    gpsWeek, days = divmod(timeSinceGpsEpoch.days, 7)
    gpsTow = (days * 86400) + timeSinceGpsEpoch.seconds + (timeSinceGpsEpoch.microseconds / 1e6)
    
    return (gpsWeek, gpsTow)

In [236]:
def gps_to_datetime(wks, tow, leaps):
    '''Convert a GPS time to a naive date time object.
        
    Receives GPS time in weeks since GPS Epoch, seconds of current week,
    leap seconds since GPS epoch.
    
    Returns datetime object.
    
    '''
    
    gpsEpoch = datetime.datetime(1980, 1, 6, 0, 0, 0)
    
    gpsWeekDays = wks * 7
    gpsTowDays, gpsTowSeconds = divmod(tow, 86400)
    
    gpsDays = int(gpsWeekDays + gpsTowDays)
    gpsSeconds = int(gpsTowSeconds)
    gpsMicroseconds = int((gpsTowSeconds - gpsSeconds) * 1e6)
    
    utcTime = gpsEpoch + datetime.timedelta(days=gpsDays, seconds=gpsSeconds, microseconds=gpsMicroseconds)
    
    # adjust for leap seconds
    utcTime -= datetime.timedelta(seconds=leaps)
    
    return utcTime

# Testing

### Test 1

In [237]:
time1 = gps_to_datetime(2053, 136137.268193, 18)
time1

datetime.datetime(2019, 5, 13, 13, 48, 39, 268192)

In [238]:
gps1 = datetime_to_gps(time1, 18)
gps1

(2053, 136137.268192)

### Test 2

In [239]:
time2 = datetime.datetime.now()
print(time2.strftime(fmt))

2019-05-17, 15:03:06.131536


In [240]:
gps2 = datetime_to_gps(time2, 18)
gps2

(2053, 486204.131536)

In [241]:
gps_to_datetime(gps2[0], gps2[1], 18)

datetime.datetime(2019, 5, 17, 15, 3, 6, 131536)

In [242]:
print(gps_to_datetime(gps2[0], gps2[1], 18).strftime(fmt))

2019-05-17, 15:03:06.131536


### General

In [244]:
# convert GPS time to UTC (naive)
gps_to_datetime(2052, 247709.485292, 18).strftime(fmt)

'2019-05-07, 20:48:11.485291'

In [245]:
# convert unix timestamp to GPS time
datetime_to_gps(datetime.datetime.fromtimestamp(1557262091.519499), 18)

(2052, 251309.519499)

In [246]:
# print date time of unix timestamp
datetime.datetime.fromtimestamp(1557262091.519499).strftime(fmt)

'2019-05-07, 21:48:11.519499'

In [247]:
# convert GPS time to unix timestamp
datetime.datetime.timestamp(gps_to_datetime(2052, 247709.485292, 18))

1557258491.485291

In [249]:
# difference between a unix timestamp and a GPS time
unixTime = 1557262091.674614
gpsTime = (2052, 247709.635292)
unixTime - datetime.datetime.timestamp(gps_to_datetime(gpsTime[0], gpsTime[1], 18))

3600.0393228530884